### Run
Only run retriever evaluator on Jupyter Notebook

In [1]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.parent))

from source.rag.retrieval import RetrievalPipeline
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.core.evaluation import (
    EmbeddingQAFinetuneDataset,
)
import nest_asyncio
nest_asyncio.apply()

In [2]:
metrics = ['mrr']
retriever = RetrievalPipeline()

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=retriever, workers=5
)

ic| f'Collection {self.index_name} have existed': 'Collection trduy_test2 have existed'
/workspace/competitions/Sly/Duy_NCKH_2025/source/database/qdrant.py:51: UserWarning: Qdrant client version 1.14.2 is incompatible with server version 1.11.3. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  self.client = QdrantClient(url, timeout=60)
ic| Settings: _Settings(_llm=OpenAI(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7fcc1c8b0d30>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7fcc22d82320>, completion_to_prompt=<function default_completion_to_prompt at 0x7fcc22c1a3b0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='gpt-4o-mini', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers={}, reuse_client=T

In [3]:
qa_dataset = EmbeddingQAFinetuneDataset.from_json("/workspace/competitions/Sly/Duy_NCKH_2025/data/retrieval_evaluation/qa_dataset2.json")

In [6]:
# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[2]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

Query: Nếu tôi điều khiển xe chở hàng vượt trọng tải cho phép từ 100% đến 150%, tôi sẽ bị phạt bao nhiêu tiền?
Metrics: {'mrr': 1.0}



In [7]:
# try it out on an entire dataset
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset, show_progress=True)

  0%|          | 0/628 [00:00<?, ?it/s]

100%|██████████| 628/628 [43:19<00:00,  4.14s/it]    


In [9]:
import json

def save_eval_results_to_json(eval_results, output_path: str):
    simplified_results = []

    for result in eval_results:
        simplified = {
            "query": result.query,
            "expected_ids": result.expected_ids,
            "expected_texts": result.expected_texts,
            "retrieved_ids": result.retrieved_ids,
            "retrieved_texts": result.retrieved_texts,
            "score": result.metric_dict["mrr"].score # Lấy điểm MRR nếu có
        }
        simplified_results.append(simplified)


    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(simplified_results, f, indent=2, ensure_ascii=False)

output_path = '/workspace/competitions/Sly/Duy_NCKH_2025/data/retrieval_evaluation/experiment_2/qa_result2.json'
save_eval_results_to_json(eval_results, output_path)

In [10]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

In [11]:
display_results("with no reranking", eval_results)

,retrievers,mrr
0,with no reranking,0.601166


In [12]:
print(eval_results[0])

Query: Nếu tôi điều khiển xe mà không có giấy chứng nhận kiểm định an toàn kỹ thuật còn hiệu lực, tôi sẽ bị phạt bao nhiêu tiền và có thể bị xử lý như thế nào?
Metrics: {'mrr': 0.2}

